<a href="https://colab.research.google.com/github/megan-the-astronomer/ASTR229/blob/main/functions_functionality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packaging code, file i/o, and putting the pieces together

This week, we'll look at a few more skills to shore up your toolkit for the coding challenges ahead.

First, we'll look at functions which are a great way to make your code more readable and reusable.

Next, we'll look at some simple tools for reading/writing files (you already have some experience with this from an earlier notebook where you read in images and headers from a fits file).

Let's do this.  

In [ ]:
# start by importing a few essentials
import numpy as np
import matplotlib.pyplot as plt
import astropy

from astropy.io import fits
from astropy.table import Table

# Functions

Recommended reading: https://philuttley.github.io/prog4aa/12-func/index.html

Functions make code more readable, easier to package and reuse. Functions contain blocks of code that you are likely to use over and over again.

Let's start with a simple but useful example - converting the temperature from celsius to kelvin.

In [ ]:
def celsius_to_kelvin(temp_c):
    return temp_c + 273.15

The ```def``` keyword signifies the start of a function definition.

This is followed by the name of the function (```celsius_to_kelvin```) and a list of parameter names in parentheses (only one in this function - ```temp```). Note the colon (```:```) at the end of the line.

The code to be executed starts on the next line - note that these lines of code are indented!

The last line of the function starts with the ```return``` keyword followed by the value to be returned by the function.

Values passed to the function - in this case the specific temperature to convert - are assigned to variables in the function (```temp``` in this case).

Let’s try running our function.

In [ ]:
# The average temperature of the Earth is ~15 C
celsius_to_kelvin(15.)

288.15

This calls the function using ```15.``` as the input and returns the converted temperature.

You can use the output in a print statement or assign it to a variable. Functions that you write yourself behave like other functions you have used in python already (e.g., ```print()``` or ```plt.plot()```).  

In [ ]:
print('Approximate temperature of the Earth in Kelvin:', celsius_to_kelvin(25.))

Approximate temperature of the Earth in Kelvin: 298.15


In [ ]:
T_earth = celsius_to_kelvin(15.)

In [ ]:
T_earth

288.15

## Composing functions

Functions can save you from repeating lines of code to accomplish the same task. Copying and pasting not only makes code very long (and harder to read) but also makes it too easy to create problems for yourself (e.g., accidently overwriting a variable).

You can streamline your code by combining functions inside other functions. To borrow the example from [Phil Uttley](https://philuttley.github.io/prog4aa/12-func/index.html), you can easily convert temperature from Fahreheit to Kelvin using existing functions to convert from Fahrenheit to Celsius and Celsius to Kelvin.

In [ ]:
def fahr_to_celsius(temp):
    return ((temp - 32) * (5/9))

In [ ]:
def fahr_to_kelvin(temp_f):
    temp_c = fahr_to_celsius(temp_f)
    temp_k = celsius_to_kelvin(temp_c)
    return temp_k

In [ ]:
# daily highs in Houston this week have been ~90 F
print('Houston daytime highs in Kelvin:', fahr_to_kelvin(90.0))

Houston daytime highs in Kelvin: 305.3722222222222


This simple example illustrates a few key points. Programs are typically built by combining separate, smaller chunks of code. While most programs are longer than 2-3 lines, ideally they are not 500 lines long either as this makes the code harder to read and harder to debug (that is, identify and troubleshoot problems).

# A few more notes on functions

Functions don't always have return statements at the end. For example, to look through the McDonald data of M82, you might write a function to create an image of every frame with a title that reports the exposure time and the filter or band used to take the data. Keep this idea in mind when writing longer code for a project - generating diagnostic plots along the way can be a great way to make sure intermediate steps have executed correctly.

Just like with variable names, it is highly recommended to give functions meaningful names that make your code more human readable.

# Testing, debugging, and documenting

Part of the power of functions is that they are meant to be reusable. To make sure that is true, it's important to test the code.

The first test should be to start with something where you know the answer. If the code does not produce the correct answer, you know you have a problem.

Next, try a few different test cases. In the example above, make sure that you get the correct answer for both positive and negative temperatures.

Let's try an example. Use the data tables from last week to predict the apparent magnitude of a star.

In [ ]:
def predict_apparent_mag(abs_mag, dist):
  mag = abs_mag + 5 * np.log10(dist) - 5
  return mag

The Sun is an easy test - we can look up the spectral type of the Sun (G5) and the distance ($\sim 1.4 \times 10^{13}$ cm). From class, we know that the apparent magnitude of the Sun is about $-26.5$ mag.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dat = Table.read('/content/drive/MyDrive/ASTR229_data/stellar_data/ms.dat', format='ascii.csv', delimiter=';', comment='#')

In [ ]:
# find the solar type star
solar_type = np.where(dat['Spectral Type'] == 'G5')
dat[solar_type[0]]

Spectral Type,Temperature (K),Absolute Magnitude,Luminosity (in solar luminosities)
str2,int64,str5,float64
G5,5660,4.9,0.86


In [ ]:
# we need the distance in the Sun in pc but the number we have is in cm
dsun_cm = 1.4e13 # cm
pc_cm = 3.1e18 # cm
dsun_in_pc = dsun_cm / pc_cm

In [ ]:
# the Sun is close so the distance should be a tiny fraction of a pc
dsun_in_pc

4.516129032258065e-06

In [ ]:
predict_apparent_mag(dat['Absolute Magnitude'][solar_type[0]], dsun_in_pc)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U5'), dtype('float64')) -> None

Ooops - we got an error message. What went wrong?

Python provides a lot of information that can sometimes be difficult to parse.

Starting from the bottom, we see a type error - the data type of one of our inputs doesn't match what the function was expecting.

Look a few lines above that and notice the green arrow. This points to the line where the code ran into a problem.
In this case, it's easy because the function has only one line.

For this simple mathematical calculation, the likely culprit is that one of our inputs isn't actually an integer or a float.

In [ ]:
type(dat['Absolute Magnitude'][solar_type[0]][0])

numpy.str_

Indeed, the absolute magnitude that we passed to the program is a string, not a float.

In [ ]:
# convert str to float
abs_mag_sun = float(dat['Absolute Magnitude'][solar_type[0]])

/tmp/ipython-input-1675701606.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  abs_mag_sun = float(dat['Absolute Magnitude'][solar_type[0]])


In [ ]:
predict_apparent_mag(abs_mag_sun, dsun_in_pc)

np.float64(-26.826168290780174)

Tests like this are also a great way to catch small, painful errors like typos and sign errors.

Finally, to make your code more readable for yourself and others, it's a good idea to include some documentation. We've done this for a few lines using the comment character (```#```). When we defined ```dsun_cm``` we added a comment to remind ourselves of the units.

You can also use triple quotes for larger comment blocks. If these are the first thing in your function, python attaches the comment as the documentation for the function.

In [ ]:
def predict_apparent_mag(abs_mag, dist):
  '''This function predicts the apparent magnitude of a star with a given
  absolute magnitude at some distance d (in pc!)'''
  mag = abs_mag + 5 * np.log10(dist) - 5
  return mag

In [ ]:
help(predict_apparent_mag)

Help on function predict_apparent_mag in module __main__:

predict_apparent_mag(abs_mag, dist)
    This function predicts the apparent magnitude of a star with a given
    absolute magnitude at some distance d (in pc!)



### Your turn

Write a function to convert RA and Dec from sexagesimal (hh:mm:ss and +/-dd:mm:ss) to decimal degrees.

A few tips:
 - be sure to include the sign for the Declination! We want the code to be usable for sources in any part of the sky.
 - don't forget the $cos(\mathrm{Dec})$...
 - be sure to test your code using coordinates for a few different stars. Not sure where to start? Look at some of the bright stars suggested in the eyepiece observing lab part 1.

In [ ]:
# your code

# Defining Defaults

There are two ways to pass parameters to a function: directly as we have done in ```predict_apparent_mag```, and by name as we did when reading in the data tables last week, e.g., ```dat = Table.read(filename, format='ascii.csv', delimiter=';', comment='#')```.

Notice that if you read in your table by typing ```dat = Table.read(filename, format='ascii.csv', delimiter=';')```, the command will still work.

Doing the same for ```predict_apparent_mag``` will not.

In [ ]:
predict_apparent_mag(abs_mag_sun)

TypeError: predict_apparent_mag() missing 1 required positional argument: 'dist'

We get an error because the function requires two inputs - the absolute magnitude and the distance.

We can avoid this problem by setting a default value when we write the function.

In [ ]:
def predict_apparent_mag(abs_mag, dist=10.0):
  '''This function predicts the apparent magnitude of a star with a given
  absolute magnitude at some distance d (in pc!)'''
  mag = abs_mag + 5 * np.log10(dist) - 5
  return mag

Now, the function will assume a distance of 10.0 pc if the user does not provide another value when they call the function.

In [ ]:
predict_apparent_mag(abs_mag_sun)

np.float64(4.9)

The function works the same as before when we supply both the absolute magnitude and the distance.

In [ ]:
predict_apparent_mag(abs_mag_sun, dsun_in_pc)

np.float64(-26.826168290780174)

Python assigns values to parameters from left to right. As long as you provide information in the right order, things should work.

You can also name the value to ensure the right value is passed to the right parameter.

In [ ]:
# you can also use the keyword to specify the value
predict_apparent_mag(abs_mag_sun, dist=dsun_in_pc)

np.float64(-26.826168290780174)

Look at the documentation to see which parameters in a function have a default value and which do not. For example, with ```Table.read()``` there is no default value for the filename or the table format, so both need to be specified when the function is called.

# Local and global variables

Not all variables are visible in all parts of a program. Variables can be local or global.

Local variables are defined and used inside a function. These are not visible in the main program. The variable ```dist``` used in ```predict_apparent_mag``` is a local variable. Try printing the value of ```dist``` in a new cell.

Global variables are defined outside of a specific function and are visible everywhere. We defined the specific distance to the Sun in parsec (```dsun_in_pc```) as a global variable. Try printing the value of ```dsun_in_pc``` in a new cell.

In [ ]:
dist

NameError: name 'dist' is not defined

In [ ]:
dsun_in_pc

4.516129032258065e-06

# Writing to and reading from files

Recommended reading: https://philuttley.github.io/prog4aa/13-simpleio/index.html

We've already explored a few astronomy-specific ways to read in data. We've used ```astropy``` tools to read in fits files (images and headers) with ```fits.read()``` and some tabular data using ```Table.read()```. Often, specific tools are the best way to read in and write out data.

However, there may be some cases where you wish to be able to write information to file from within your program. The python ```File``` function provides a simple way to read and write files. Let's look at an example.

In [ ]:
# use open() to open a new or existing file
# the 'w' says we will be writing to the file
f = open('example_files.txt', 'w')
f.write("I made a text file")
f.write("for my ASTR 229 homework")
# be sure to close the file! if not, bad things can happen
f.close()

When you run the cell above, the file will save to your local directory.

If you are doing the exercise in colab, take a look to see where your new file ended up. Notice that the file is in the virtual file system. For example, when I look at the file path, I see ```/content/example_files.txt```. This means the file will disappear at the end of my colab session unless I save a copy of it somewhere else.

To save the file in a specific folder or directory, specify the full file path.

In [ ]:
# now with full file path
f = open('/content/drive/MyDrive/ASTR229_sandbox/example_files.txt', 'w')
f.write("I made a text file")
f.write("for my ASTR 229 homework")
f.close()

Open the file using your favorite text editor. In colab, you can also open the file next to your code by double clicking on the filename.

Notice that the two lines of text we added to the file run right into each other. If we want to print each phrase on a separate line, we need to add formatting to specify the start of a new line.

In [ ]:
# now with formatting
f = open('/content/drive/MyDrive/ASTR229_sandbox/example_files.txt', 'w')
f.write("I made a text file \n")
f.write("for my ASTR 229 homework \n")
f.close()

You can also open an existing file to add more text using ```'a'``` to append to a file.

CAREFUL! If you try to add to a file using ```'w'``` you will overwrite the existing file.

In [ ]:
f = open('/content/drive/MyDrive/ASTR229_sandbox/example_files.txt', 'a')
f.write("This is the third python assignment \n")
f.close()

Use ```'r'``` to read in the contents of the file.

In [ ]:
f = open('/content/drive/MyDrive/ASTR229_sandbox/example_files.txt', 'r')
print(f.read())
f.close()

I made a text file 
for my ASTR 229 homework 
This is the third python assignment 



To read individual lines, you can use ```readline``` instead of ```read```.

In [ ]:
f = open('/content/drive/MyDrive/ASTR229_sandbox/example_files.txt', 'r')
print(f.readline())
f.close()

I made a text file 



To avoid problems with files not being closed properly, it is recommended to use the ```with``` keyword to open files.

In [ ]:
with open('another_example.txt', 'w') as f:
    f.write('O-type stars are really hot\n')
    f.write('M-type stars really are not\n')

The file is automatically closed at the end of the indented text.

The same approach works for reading in files.

In [ ]:
with open('/content/drive/MyDrive/ASTR229_data/stellar_data/giants.dat', 'r') as f:
    print(f.read())

Spectral Type; Temperature (K); Absolute Magnitude; Luminosity (in solar luminosities)
#
G5; 5010; 0.7; 127
G8; 4870; 0.6; 113
K0; 4720; 0.5; 96
K1; 4580; 0.4; 82
K2; 4460; 0.2; 70
K3; 4210; 0.1; 58
K4; 4010; 0.0; 45
K5; 3780; -0.2; 32
M0; 3660; -0.4; 15
M1; 3600; -0.5; 13
M2; 3500; -0.6; 11
M3; 3300; -0.7; 9.5
M4; 3100; -0.75; 7.4
M5; 2950; -0.8; 5.1
M6; 2800; -0.9; 3.3


This basic tool can be really useful for outputting formatted results. For example, I'm often writing formatted text files to output data that I want to put in a table in my paper.

One practical application is to use your knowledge of string manipulation and file i/o (input/output) to write a [ds9 region](https://ds9.si.edu/doc/ref/region.html) file. Recall from the first assignment that ds9 regions allow you to draw a shape on your image. You can save these regions to a formatted file so you can upload them again at a later time. You can also upload a file with a list of regions to be overplotted on your image.

#### Your turn

Make a region file to draw circles around stars near M82. We'll use the format

```
circle RA Dec radius
```

To get this to play well with ds9, we need to specify the RA and Dec in ```hh:mm:ss``` and ```+/-dd:mm:ss``` format.  

For example, to put a circle at the center of M82, my region file has the following format:

```
circle 09:55:52.4 +69:40:46 10"
```



Be sure to look at the documentation to get an idea of other formatting you can specify in your region file (e.g., the color or linewidth).

A quick way to get an idea what a region file looks like is to make one of your own in ds9. Play around with the M82 images from McDonald and DSS. Try making regions files for both. Play around with different formats for the outputs.

Now, make a region file of stars to plot on the DSS image using the ```stars_near_M82.txt``` file in the ```ASTR229_data``` folder.  